In [5]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
from datetime import datetime
import os
import glob 

In [6]:
start_time = datetime.now()

In [7]:
dir = 'Y:\\delete\\ECHA'
for f in os.listdir(dir):
    os.remove(os.path.join(dir, f))

In [8]:
def handle_popup():
    try:
        driver.find_elements(by=By.XPATH, value='//*[@id="_viewsubstances_WAR_echarevsubstanceportlet_acceptDisclaimerButton"]/span')[0].click()
    except:
        pass
    return None

In [9]:
# create webdriver object
ser = Service("C:/Users/AIMPAI0/Downloads/chromedriver.exe")
op = webdriver.ChromeOptions()
prefs = {"download.default_directory" : "Y:\\delete\\ECHA"}
op.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome(service=ser, options=op)
driver.maximize_window()  
for i in range(1,3):

    driver.get("https://echa.europa.eu/information-on-chemicals/pic/chemicals")
    driver.implicitly_wait(5)
    handle_popup()
    r =driver.find_elements(by=By.XPATH, value='//*[@id="_chemicals_WAR_echapicportlet_limitationId"]/option')
    r[i].click()
    driver.find_element(by=By.XPATH, value='//*[@id="_chemicals_WAR_echapicportlet_magnifierFieldName"]').click()
    driver.implicitly_wait(5)
    handle_popup()
    driver.implicitly_wait(5)
    handle_popup()
    driver.find_element(by=By.XPATH, value='//*[@id="_chemicals_WAR_echapicportlet_exportCsvButton"]/span').click()
    #WebDriverWait(self.driver, 5).until(EC.element_to_be_clickable((By.XPATH, self.helper.get_xpath('//*[@id="_chemicals_WAR_echapicportlet_exportCsvButton"]/span')))).click()


In [37]:
path = 'Y:\\delete\\ECHA'
filenames = glob.glob(path + "\*.csv")
df= []
for f in filenames:
    dataset = pd.read_csv(f)
    if "1" in f:
        dataset['Status'] = 'Severly Restricted'
    else:
        dataset['Status'] = 'Banned'
    df.append(dataset)
dataset = pd.concat(df, axis=0, ignore_index=True)
dataset = dataset[['CAS_No','Status']]
dataset.rename(columns = {'CAS_No':'CAS Number'}, inplace = True)
dataset = dataset.applymap(lambda x: x.strip() if isinstance(x, str) else x)

CAS = pd.read_excel('Y:/delete/Chemical CAS.xlsx')
CAS['value'] = 'Concluded'
CAS = CAS.applymap(lambda x: x.strip() if isinstance(x, str) else x)

result = pd.merge(dataset, CAS, how="outer", on="CAS Number")
result.Status.fillna(result.value, inplace=True)
del result['value']
result.head(2)


,CAS Number,Status
0,3542-36-7,Severly Restricted
1,93776-12-6,Severly Restricted


In [38]:
result['Jurisdiction'] = 'Europe'
result['RegulatoryAgency'] = 'European Chemicals Agency (ECHA)'
result['TypeOfRegulation'] = 'Health'
result['Criteria'] = 'Endocrine Disruption'

result['Description'] = 'No restriction'
result.loc[result['Status'] == 'Severly RestrictedSeverly Restricted', 'Description'] = 'Severely restricted for some applications'
result.loc[result['Status'] == 'Banned', 'Description'] = 'Banned for some applications'

result['HazardClassification'] = 'Low'
result.loc[result['Status'] == 'Severly Restricted', 'HazardClassification'] = 'High'
result.loc[result['Status'] == 'Banned', 'HazardClassification'] = 'Moderate'

result['CauseOfClassification'] = 'No use limitaion'
result.loc[result['Status'] == 'Severly Restricted', 'CauseOfClassification'] = 'Use limitation - severely restricted'
result.loc[result['Status'] == 'Banned', 'CauseOfClassification'] = 'Use limitation - banned'

result['SourceName'] = 'European Chemicals Agency'
result['SourceURL'] = 'http://echa.europa.eu/nl/ed-assessment'
result = result.drop(['Status'], axis=1)

result.head(2)

,CAS Number,Jurisdiction,RegulatoryAgency,TypeOfRegulation,Criteria,Description,HazardClassification,CauseOfClassification,SourceName,SourceURL
0,3542-36-7,Europe,European Chemicals Agency (ECHA),Health,Endocrine Disruption,No restriction,High,Use limitation - severely restricted,European Chemicals Agency,http://echa.europa.eu/nl/ed-assessment
1,93776-12-6,Europe,European Chemicals Agency (ECHA),Health,Endocrine Disruption,No restriction,High,Use limitation - severely restricted,European Chemicals Agency,http://echa.europa.eu/nl/ed-assessment


In [ ]:
results.to_csv('Y:/delete/Reugulatory Assessment.csv',index=False,mode='a',header=False)

In [39]:
end_time = datetime.now()
print('The total time taken is', (end_time - start_time))

The total time taken is 0:18:17.016489
